In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [19]:
test_df = pd.read_csv("E:/Projects/Learning/ML/Competition/Data/test.csv")
df = pd.read_csv("E:/Projects/Learning/ML/Competition/Data/train_folds.csv")
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,...,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,0
1,2,B,B,A,A,B,D,A,F,A,...,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,2
2,3,A,A,A,C,B,D,A,D,A,...,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4
3,4,B,B,A,C,B,D,A,E,C,...,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,3
4,6,A,A,A,C,B,D,A,E,A,...,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,1


In [20]:
features = [col for col in df.columns if col not in ('id', 'kfold', 'target')]
object_cols = [col for col in features if 'cat' in col]
xtest_df = test_df[features].copy()

In [21]:
final_predictions = []
for fold in range(5):
    xtrain = df.loc[df.kfold != fold].reset_index(drop=True)
    xvalid = df.loc[df.kfold == fold].reset_index(drop=True)
    xtest = xtest_df.copy()
    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[features]
    xvalid = xvalid[features]

    print("Encoding")
    ordinal_encoder = OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])

    print("Training")
    model = XGBRegressor(random_state=fold, n_jobs=8)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    preds_test = model.predict(xtest)
    final_predictions.append(preds_test)
    print(f"Fold_{fold} |", "RMSE:", mean_squared_error(preds_valid, yvalid, squared=False))

Encoding
Training
Fold_0 | RMSE: 0.7249295276475416
Encoding
Training
Fold_1 | RMSE: 0.7247982342472964
Encoding
Training
Fold_2 | RMSE: 0.726816043771052
Encoding
Training
Fold_3 | RMSE: 0.7268377871109586
Encoding
Training
Fold_4 | RMSE: 0.7257551866601156


In [24]:
preds = np.mean(np.column_stack(final_predictions), axis=1)

In [27]:
sample_submission = pd.DataFrame({'id':test_df.id, 'target':preds})
# sample_submission

,id,target
0,0,8.075241
1,5,8.320848
2,15,8.427451
3,16,8.413889
4,17,8.192506
...,...,...
199995,499987,8.091112
199996,499990,8.391352
199997,499991,8.370418
199998,499994,8.279126
